In [1]:
from personal_twitter_data import TweetLoader
from mongo_data import MongoTweetStore
from sentiment_analysis import RobertaModel
import pymongo

/Users/jasonzhang/Documents/PersonalProjects/twitter_dashboard/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = TweetLoader()
loader.extract_tweets(count=10)

In [3]:
len(loader.get_loaded_tweets_as_json())

10

In [4]:
for tweet in loader.tweets:
    print(tweet.author.name)
    # print(tweet._json)

Forbes
Forbes
The Wall Street Journal
Forbes
Forbes
Bleacher Report
Bloomberg
Forbes
Forbes
Bloomberg


In [5]:
store = MongoTweetStore("twitter_dashboard_db", "home_timeline")

In [6]:
store.collection.create_index([("id", pymongo.ASCENDING)], name="tweet_id", unique=True, default_language="english")

'tweet_id'

In [7]:
roberta = RobertaModel()
store.save_tweets_to_db(loader.tweets, model=roberta)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1. business
2. Forbes
3. Forbes
4. business
5. BleacherReport
6. Forbes
7. Forbes
8. WSJ
9. Forbes
10. Forbes


In [ ]:
data[0].keys()

In [ ]:
data = store.load_data_test(users=["business", "Forbes"], n_tweets=100, n_user_tweets=100)

In [8]:
from mongo_data import MongoTweetStore
store = MongoTweetStore("twitter_dashboard_db", "home_timeline")

# Data is a list of dicts (user -> list(tweets))
u = ["business", "Forbes"]
# data = store.load_data_test(users=u, n_tweets=200, n_user_tweets=10, latest=False)
data = store.load_data_test(groupby_user=False)
[t["created_at"] for t in data["business"]]
data

# d = []
# for group in data:
#     d.append(group)
#     # print(type(group))
#     print(group.keys())

{'BleacherReport': [{'_id': ObjectId('64064d9a414a3f592e3cac44'),
   'created_at': 'Mon Mar 06 20:26:46 +0000 2023',
   'id': 1632840165248098304,
   'id_str': '1632840165248098304',
   'text': 'The NBA has rescinded Giannis’ 10th rebound last night, which gave him a triple-double, per @ZachLowe_NBA\n\nLeague r… https://t.co/mP8q9PiTk6',
   'truncated': True,
   'entities': {'hashtags': [],
    'symbols': [],
    'user_mentions': [{'screen_name': 'ZachLowe_NBA',
      'name': 'Zach Lowe',
      'id': 23378774,
      'id_str': '23378774',
      'indices': [92, 105]}],
    'urls': [{'url': 'https://t.co/mP8q9PiTk6',
      'expanded_url': 'https://twitter.com/i/web/status/1632840165248098304',
      'display_url': 'twitter.com/i/web/status/1…',
      'indices': [117, 140]}]},
   'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
   'in_reply_to_status_id': None,
   'in_reply_to_status_id_str': None,
   'in_reply_to_user_id': None,
   'in_reply_

In [ ]:
type(d[0]["docs"][0])

In [ ]:
data = store.load_data(latest=True)

for user_data in data:
    print(user_data["user"])
    print(len(user_data["tweets"]))
    for tweet in user_data["tweets"]:
        print(tweet["created_at"])

In [ ]:
from pprint import pprint

def print_documents(cursor):
    for document in cursor:
        pprint(document)
        # print(document["user"]["screen_name"])
        # print(document["id"])
        # print(document["text"])

# store.collection.aggregate(["$match": {"$in": ["WSJ"]}])
data = store.collection.find({"user.screen_name": {"$in": ["WSJ", "business"]}})
print_documents(data)


In [ ]:
users = ["WSJ", "business", "NASA"]
n_tweets = 1000
n_user_tweets = 1000
num = 2
latest = False
pipeline = [
    # Filter only requested users
    # {"$match": {"user.screen_name": {"$in": users}}},
    # Group tweets by user
    {
        "$group": {
            "_id": "$user.screen_name",
            "docs": {"$push": "$$ROOT"}
            },
    },
]

data = store.collection.aggregate(pipeline)
# print_documents(data)

# Order alphabetically by name (Mongo doesn't return in specific order)
data = sorted(data, key=lambda x: x["_id"].lower())

tweets_count = 0
for group in data:
    if tweets_count >= n_tweets:
        break

    tweets = group["docs"]
    print(group["_id"])

    if len(tweets) > n_tweets - tweets_count:
        tweets = tweets[: n_tweets - tweets_count]

    if n_user_tweets:
        tweets = tweets[:n_user_tweets]
        # print(group["docs"])
        # print(len(group["docs"]))
        print(len(tweets))

    tweets_count += len(tweets)